# RSS reader

## References

- https://www.w3schools.com/xml/xml_rss.asp
- https://towardsdatascience.com/how-i-turned-my-companys-docs-into-a-searchable-database-with-openai-4f2d34bd8736


In [5]:
# dependencies
%pip install feedparser openapi python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
# config
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Feed

First, we need to import the RSS feeds.
There is a top level `channel`, with:

- `title`
- `link`
- `description`
- `category`: optional
- `copyright`: optional
- `image`: optional, has `url`, `title`, `link`
- `language`: optional
- `pubDate`: optional
- a list of `items`

Each item has:

- `title`
- `link`
- `description`
- `author`: optional
- `comments`: optional
- `pubDate`: optional
- `enclosure`: optional, `url`, `length`, `type` attributes


In [20]:
import feedparser

feeds = ['https://news.ycombinator.com/rss', 'https://themacrocompass.substack.com/feed']
for f in feeds:
    feed = feedparser.parse(f)
    print('-----')
    print('title:', feed.feed.title)
    print('link: ',feed.feed.link)
    print('-----')
    for e in feed.entries:
        print('title:', e.get('title', ''))
        print('link:', e.get('link', ''))
        print('description:', e.get('description', ''))
        print('pubDate:', e.get('pubDate', ''))
        print('comments:', e.get('comments', ''))
        print('---')

-----
title: Hacker News
link:  https://news.ycombinator.com/
-----
title: Bitsavers Software Archive
link: https://bitsavers.org/
description: <a href="https://news.ycombinator.com/item?id=36477233">Comments</a>
pubDate: no_pubDate
comments: https://news.ycombinator.com/item?id=36477233
---
title: Why Britain Doesn’t Build
link: https://worksinprogress.co/issue/why-britain-doesnt-build
description: <a href="https://news.ycombinator.com/item?id=36477481">Comments</a>
pubDate: no_pubDate
comments: https://news.ycombinator.com/item?id=36477481
---
title: Mycelite: SQLite extension to synchronize changes across SQLite instances
link: https://github.com/mycelial/mycelite
description: <a href="https://news.ycombinator.com/item?id=36475081">Comments</a>
pubDate: no_pubDate
comments: https://news.ycombinator.com/item?id=36475081
---
title: Show HN: WinGPT – AI assistant for Windows 3.1
link: https://www.dialup.net/wingpt/
description: <a href="https://news.ycombinator.com/item?id=36472854">Co

## API

We use the OpenAI API to summarize the relevant documents.


In [7]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an assistant which reads and summarizes articles."},
    {"role": "user", "content": "Summarize this: https://ohwr.org/project/white-rabbit/wikis/home"}
  ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "The White Rabbit project is an effort to develop a new synchronization and deterministic data transfer technology for high-precision time and frequency distribution in large scientific projects like CERN's LHC accelerator. White Rabbit uses Synchronous Ethernet-based network and a high-precision time and frequency distribution protocol (PTP) to achieve 1 ns accuracy for time transfer and 1 ps accuracy for frequency transfer, with sub-ns precision in both cases. The technology is being developed as an open-source project, with the goal of making it widely available to the scientific community and beyond."
}
